In [ ]:
!pip install PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 47.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os

pdf_dir = '/content/drive/My Drive/UCF/BogenMaterials'

pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith('.pdf')]
print("Found PDF files:", pdf_files)

import fitz  # PyMuPDF
e700_manual = '/content/drive/My Drive/UCF/BogenMaterials/E7000 Overview Spec Sheet.pdf'

# Open PDF file
doc = fitz.open(e700_manual)
text = ""
for page in doc:
    text += page.get_text()

Found PDF files: ['E7000 Overview Spec Sheet.pdf', 'Catalog 112- Ron Copy ISC West.pdf', 'bogen_catalog_111_0.pdf', 'NyquistSystemAdministratorManual.pdf', 'Nyquist E700MCR - Quick Start Guide.pdf', 'Nyquist-E7000-Site-Survey.pdf', 'Nyquist_E7000_System_Controller_Setup_Guide.pdf', 'Nyquist_E7000_Server_Installation_and_Setup_Guide.pdf', 'Nyquist-E7000-System-Administrator-Manual-V5.pdf']


In [ ]:
!pip install transformers


In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [53]:
import fitz  # PyMuPDF
import re

# Path to your PDF file in Google Drive
pdf_path = '/content/drive/My Drive/UCF/BogenMaterials/bogen_catalog_111_0.pdf'

# Open the PDF
doc = fitz.open(pdf_path)

# Set the page from which to start extracting text (e.g., skip the first 3 pages)
start_page = 4  # Adjust this based on the actual location of the TOC

# Extract text from pages after the TOC
text = ""
for page_num in range(start_page, doc.page_count):  # Start from the specified page
    page = doc[page_num]
    text += page.get_text()

# Display the first part of the extracted text for inspection
print("Extracted Text:\n", text[:100])

# Example pattern to split by headings (adjust for your text format)
sections = re.split(r'(?:\n|\r\n)[A-Z ]+\n', text)  # Adjust regex to match headers

# Step 4: Print the first few sections to inspect
print("First few sections of the extracted text:")
for i, section in enumerate(sections[:5]):  # Print the first 5 sections
    print(f"Section {i + 1}:\n{section}\n")


Extracted Text:
 NYQUIST System Controller 
Model NQ-SYSCTRL
Bogen’s Nyquist System Controller offers contractors a c
First few sections of the extracted text:
Section 1:
NYQUIST System Controller 
Model NQ-SYSCTRL
Bogen’s Nyquist System Controller offers contractors a cost-effective means to 
deploy the Nyquist C4000 solution using a state-of-the-art processing platform 
pre-installed with the operating system and C4000 application software. The Nyquist 
System Controller facilitates quick installs and offers high-performance operation for 
even the largest Nyquist C4000 system configurations.
P R O D U C T  F E A T U R E S :
• Configuration and management via a web-based graphical user interface
• Remotely administrable from any PC/Mac, tablet, or mobile device
• Wizard-based Setup Assistant for quick and easy installation
• High-performance design capable of supporting even the largest C4000 applications
• Dual RJ45 10/100/1000 Mbps NICs for separate C4000 network and Internet acces

In [ ]:
# Step 5: Load a Hugging Face Model for Question Generation
qg_pipeline = pipeline("text2text-generation", model="google/flan-t5-large")

In [ ]:
from transformers import pipeline

# Load the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


In [56]:
# Summarize each section with additional checks
summaries = []
for section in sections:
    # Skip empty, short, or non-text sections
    if len(section.strip()) < 20 or not re.search(r'[a-zA-Z]', section):
        continue

    # Limit section length to avoid overly large inputs
    section = section[:1000]  # Limit to 1000 characters if needed

    try:
        # Generate a summary for each valid section
        summary = summarizer(section, max_length=50, min_length=25, do_sample=False)[0]["summary_text"]
        summaries.append(summary)
    except Exception as e:
        print(f"Error summarizing section: {section[:100]}... - {str(e)}")
        continue

# Print the first few summaries for inspection
print("First few summaries:")
for i, summary in enumerate(summaries[:5]):
    print(f"Summary {i + 1}:\n{summary}\n")

Your max_length is set to 50, but your input_length is only 18. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)
Your max_length is set to 50, but your input_length is only 8. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)
Your max_length is set to 50, but your input_length is only 6. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)
Your max_length is set to 50, but your input_length is only 6. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)
Your max_length

First few summaries:
Summary 1:
Bogen’s Nyquist System Controller offers contractors a cost-effective means to deploy the Nyquist C4000 solution using a state-of-the-art processing platform. The Nyquist                 System Controller facilitates quick installs and

Summary 2:
Bogen’s Nyquist-based networked audio power amplifiers offer cost-effective performance for permanent installations and commercial sound. They offer state-of-the-art, built-in DSP software for extensive and 

Summary 3:
Bogen’s Nyquist networked Public Address Mixer/Amplifier (PAMA) series offers cost-effective performance for permanent installations and commercial sound. PAMA series provides unparalleled flexibility and versatility without the

Summary 4:
Bogen’s Nyquist networked 4-Channel Matrix Mixer Pre-Amplifier (MMPA) offers cost- and cost-effective performance for permanent installations and commercial sound. The MMPA provides unparalleled flexibility and 

Summary 5:
Bogen’s plenum-rated VoIP intercom 

In [58]:
# Generate Q&A pairs based on summaries
qa_pairs = []
for summary in summaries:
    # Generate a question based on the summary
    prompt = f"Generate a question to which the answer would be the following summary: {summary}"
    question = qg_pipeline(prompt, max_length=50, num_return_sequences=1)[0]["generated_text"]

    # Append question and summary as answer
    qa_pairs.append({"question": question, "answer": summary})

# Print the first few Q&A pairs for inspection
print("\nGenerated Q&A pairs:")
for qa in qa_pairs[:5]:
    print("Q:", qa["question"])
    print("A:", qa["answer"])
    print()


Generated Q&A pairs:
Q: What is the name of Bogen's system controller?
A: Bogen’s Nyquist System Controller offers contractors a cost-effective means to deploy the Nyquist C4000 solution using a state-of-the-art processing platform. The Nyquist                 System Controller facilitates quick installs and

Q: what is the name of the company that makes the bogen nyquist-based networked audio power amplifiers?
A: Bogen’s Nyquist-based networked audio power amplifiers offer cost-effective performance for permanent installations and commercial sound. They offer state-of-the-art, built-in DSP software for extensive and 

Q: What is the name of Bogen's public address mixer/amplifier series?
A: Bogen’s Nyquist networked Public Address Mixer/Amplifier (PAMA) series offers cost-effective performance for permanent installations and commercial sound. PAMA series provides unparalleled flexibility and versatility without the

Q: What is the name of Bogen's matrix mixer?
A: Bogen’s Nyquist netwo

In [60]:
import json

# Define the output path
output_path = '/content/drive/My Drive/UCF/BogenMaterials/qa_dataset2.json'

# Save Q&A pairs to JSON
with open(output_path, "w") as f:
    json.dump(qa_pairs, f, indent=4)

print("Q&A pairs saved to", output_path)


Q&A pairs saved to /content/drive/My Drive/UCF/BogenMaterials/qa_dataset2.json
